In [78]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import csv

In [79]:
url = 'https://www.imdb.com/list/ls062911411/?sort=user_rating,desc&st_dt=&mode=detail&page=1'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
# response    --> <Response [200]> means it is successful
# print(soup) --> prints the html of the entire page.

In [80]:
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
link = []
descriptions = []
genres = []

In [81]:
# When you hover over the movie and click inspect, you can see that the movie information is stored in this div class.
movie_data = soup.findAll('div', attrs = {'class' : 'lister-item mode-detail'}) 
# movie_data --> I can see movie details inside div class

In [82]:
csv_filename = "movies_data.csv"
with open(csv_filename, mode = "w", newline = '', encoding = 'utf-8-sig') as file:
  writer = csv.writer(file)

  header = ['Name of movie','Year of relase','Watchtime','Movie rating','Metascore','Votes','Gross','Link']
  writer.writerow(header)

  for point in movie_data :
    name = point.h3.a.text
    movie_name.append(name)
    
    new_link = 'https://www.imdb.com/' +point.h3.a.get('href')
    link.append(new_link)
    new_url = requests.get(new_link)
    new_soup = BeautifulSoup(new_url.content, 'html.parser')
    
    #movie_details = new_soup.findAll('div', attrs = {'class' :'sc-e226b0e3-6 CUzkx'})
    #for i in movie_details:
      #genre = i.find('div', class_='ipc-chip-list__scroller')
      #genres.append(genre)
      
    #for i in movie_details:
    #  description_tag = i.find('p', class_='sc-466bb6c-0 hlbAws')
    #  description = description_tag.text.strip() if description_tag else 'No description available.'
    #  descriptions.append(description)

    #movie_details = new_soup.find('div', {'class': 'ipc-page-content-container ipc-page-content-container--center'})
    #description_tag = movie_details.find('p', class_='sc-466bb6c-3 fOUpWp')
    #description = description_tag.text.strip() if description_tag else 'No description available.'
    #descriptions.append(description)

    #if movie_details:
    #        description_tag = movie_details.find('p', class_='sc-466bb6c-3 fOUpWp')
    #        description = description_tag.text.strip() if description_tag else 'No description available.3'
    #        descriptions.append(description)
    #else:
    #    descriptions.append('No description available.1')            

  # When you hover over the movie and click inspect, you can see that the movie information is stored in this span class.
  # If we do not write text.replace, the result will be like this: [<span class="lister-item-year text-muted unbold">(1962)</span>]
    release_year = point.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
    year.append(release_year)

    run_time = point.p.find('span', class_ = 'runtime').text.replace(' min', '')
    time.append(run_time)

    rate = point.find('div', class_ = 'ipl-rating-star small').text.replace('\n','')
    rating.append(rate)

    meta = point.find('span', class_ = 'metascore').text.replace(' ', '') if point.find('span', class_ = 'metascore') else '^^^^^^'
    metascore.append(meta)

    value = point.find_all('span', attrs = {'name' : 'nv'})
    vote = value[0].text.replace(' ', '')
    votes.append(vote)

    grosses =  value[1].text if len(value) > 1 else '*****'
    gross.append(grosses)

    writer.writerow([name,release_year,run_time,rate,meta,vote,grosses,new_link])


  

In [83]:
# NumPy library to find the number of non-zero elements in a NumPy array
# np.count_nonzero(movie_name) --> 100
# release_year                 --> '1962'
# year                         --> Prints all years of the first 100 movies.
# time
# rating
# metascore
# votes
# gross
# link

In [84]:
movie_DF = pd.DataFrame({'Name of movie' : movie_name, 'Year of relase' : release_year, 'Watchtime' : time, 'Movie rating' : rating, 'Metascore' : metascore, 'Votes' : votes, 'Gross' : gross, 'Link': link})

In [85]:
movie_DF
# descriptions
# genres


,Name of movie,Year of relase,Watchtime,Movie rating,Metascore,Votes,Gross,Link
0,The Shawshank Redemption,1962,142,9.3,82,"2,847,626",$28.34M,https://www.imdb.com//title/tt0111161/
1,The Godfather,1962,175,9.2,100,"1,984,212",$134.97M,https://www.imdb.com//title/tt0068646/
2,The Dark Knight,1962,152,9,84,"2,828,876",$534.86M,https://www.imdb.com//title/tt0468569/
3,The Lord of the Rings: The Return of the King,1962,201,9,94,"1,950,728",$377.85M,https://www.imdb.com//title/tt0167260/
4,Schindler's List,1962,195,9,95,"1,430,397",$96.90M,https://www.imdb.com//title/tt0108052/
...,...,...,...,...,...,...,...,...
95,Vertigo,1962,128,8.3,100,"422,342",$3.20M,https://www.imdb.com//title/tt0052357/
96,Toy Story 3,1962,103,8.3,92,"882,519",$415.00M,https://www.imdb.com//title/tt0435761/
97,Metropolis,1962,153,8.3,98,"183,304",$1.24M,https://www.imdb.com//title/tt0017136/
98,The Apartment,1962,125,8.3,94,"193,859",$18.60M,https://www.imdb.com//title/tt0053604/
